# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2021-01-01	Systematizing confidence in open research and evidence (SCORE)	SocArXiv	This paper discusses the SCORE framework to enhance confidence in open research and evidence.	Alipourfard, Nazanin et al. (2021). "Systematizing confidence in open research and evidence (SCORE)." <i>SocArXiv</i>.	systematizing-confidence-open-research	http://academicpages.github.io/files/systematizing_score.pdf	http://academicpages.github.io/files/slides_score.pdf
2022-01-01	Cultural value resonance in folktales: A transformer-based analysis with the world value corpus	International Conference on Social Computing, Behavioral-Cultural Modeling and Prediction and Behavior Representation in Modeling and Simulation	This paper uses transformer models to analyze cultural value resonance in folktales.	Benkler, Noam et al. (2022). "Cultural value resonance in folktales: A transformer-based analysis with the world value corpus." <i>International 

## Import pandas

We are using the very handy pandas library for dataframes.

In [11]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2021-01-01,Systematizing confidence in open research and ...,SocArXiv,This paper discusses the SCORE framework to en...,"Alipourfard, Nazanin et al. (2021). ""Systemati...",systematizing-confidence-open-research,http://academicpages.github.io/files/systemati...,http://academicpages.github.io/files/slides_sc...
1,2022-01-01,Cultural value resonance in folktales: A trans...,"International Conference on Social Computing, ...",This paper uses transformer models to analyze ...,"Benkler, Noam et al. (2022). ""Cultural value r...",cultural-value-resonance-folktales,http://academicpages.github.io/files/cultural_...,http://academicpages.github.io/files/slides_cu...
2,2023-01-01,Mapping a Plurality of Explanations with NLP: ...,Proceedings of the Annual Meeting of the Cogni...,This study uses NLP to map explanations provid...,"Friedman, Scott et al. (2023). ""Mapping a Plur...",mapping-plurality-explanations-nlp,http://academicpages.github.io/files/mapping_e...,http://academicpages.github.io/files/slides_ma...
3,2023-12-01,Assessing LLMs for moral value pluralism,arXiv preprint arXiv:2312.10075,This paper evaluates LLMs for their capability...,"Benkler, Noam et al. (2023). ""Assessing LLMs f...",assessing-llms-moral-value-pluralism,http://academicpages.github.io/files/assessing...,http://academicpages.github.io/files/slides_as...
4,2024-01-01,Is from ought? A comparison of unsupervised me...,Journal of Computational Social Science,This article compares unsupervised methods for...,"Brugnone, Nathan et al. (2024). ""Is from ought...",is-from-ought,http://academicpages.github.io/files/is_from_o...,http://academicpages.github.io/files/slides_is...
5,2024-01-01,Recognizing Value Resonance with Resonance-Tun...,Proceedings of the 2024 Joint International Co...,This paper explores recognizing value resonanc...,"Benkler, Noam et al. (2024). ""Recognizing Valu...",recognizing-value-resonance-robberta,http://academicpages.github.io/files/recognizi...,http://academicpages.github.io/files/slides_re...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2021-01-01-systematizing-confidence-open-research.md
2022-01-01-cultural-value-resonance-folktales.md
2023-01-01-mapping-plurality-explanations-nlp.md
2023-12-01-assessing-llms-moral-value-pluralism.md
2024-01-01-is-from-ought.md
2024-01-01-recognizing-value-resonance-robberta.md
2024-02-17-paper-title-number-4.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
category: manuscripts
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---

The contents above will be part of a list of publications, if the user clicks the link for the publication than the contents of section will be rendered as a full page, allowing you to provide more information about the paper for the reader. When publications are displayed as a single page, the contents of the above "citation" field will automatically be included below this section in a smaller font.